## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-07-10-12-58-58 +0000,bbc,Five things Trump should know about Liberia an...,https://www.bbc.com/news/articles/c5y72w17vpwo
1,2025-07-10-12-53-00 +0000,wsj,U.S. Jobless Claims Fell Last Week,https://www.wsj.com/economy/jobs/u-s-jobless-c...
2,2025-07-10-12-50-49 +0000,nyt,K-Pop Singer Taeil Sentenced to 3½-Years in Ra...,https://www.nytimes.com/2025/07/10/world/asia/...
3,2025-07-10-12-44-50 +0000,bbc,"Ban some foreigners from sickness benefits, To...",https://www.bbc.com/news/articles/c80p1gdvyggo
4,2025-07-10-12-44-32 +0000,nyt,How El Salvador Is Reaping Rewards From Trump’...,https://www.nytimes.com/2025/07/10/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2342/2741963311.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
1,trump,16
177,will,5
57,tariffs,5
146,tariff,4
92,new,4


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
59,2025-07-09-16-39-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...,44
34,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...,39
9,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...,34
36,2025-07-10-01-07-30 +0000,bbc,Trump threatens Brazil with 50% tariff and dem...,https://www.bbc.com/news/articles/c784ee81y4zo,32
32,2025-07-10-04-00-09 +0000,nyt,His Start-Up May Not Survive Chaotic Rollout o...,https://www.nytimes.com/2025/07/10/business/tr...,30


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
59,44,2025-07-09-16-39-00 +0000,wsj,Businesses hoping for clarity on where Trump t...,https://www.wsj.com/economy/trade/trump-tariff...
34,18,2025-07-10-01-17-00 +0000,wsj,A brewing debate inside the Federal Reserve ov...,https://www.wsj.com/economy/central-banking/wh...
61,16,2025-07-09-16-28-07 +0000,nyt,Israel’s Deadly Assault on Iran Prison Incites...,https://www.nytimes.com/2025/07/06/world/middl...
49,16,2025-07-09-20-25-00 +0000,wsj,Minutes from the Fed’s June policy meeting tea...,https://www.wsj.com/economy/central-banking/ju...
14,15,2025-07-10-11-09-28 +0000,bbc,Back-to-back floods in New Mexico and Texas wi...,https://www.bbc.com/news/articles/c5y022wp009o
9,13,2025-07-10-12-04-11 +0000,nyt,"Trump Pledges 50% Tariffs Against Brazil, Citi...",https://www.nytimes.com/2025/07/09/world/ameri...
22,13,2025-07-10-09-22-00 +0000,wsj,"Tariff Anxiety Spreading Across Japan, BOJ Rep...",https://www.wsj.com/world/asia/tariff-anxiety-...
29,12,2025-07-10-05-25-38 +0000,bbc,Reform leader says council first to scrap net ...,https://www.bbc.com/news/articles/c3vde31lwpdo
26,12,2025-07-10-08-07-40 +0000,bbc,Chris Mason: Migrant deal will be seen as fail...,https://www.bbc.com/news/articles/cly8mk006kvo
25,11,2025-07-10-08-11-18 +0000,nyt,China Surveys Seabeds Where Naval Rivals May O...,https://www.nytimes.com/2025/07/10/world/asia/...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
